## Setup

In [1]:
# Make sure we're using the latest version
import Pkg
Pkg.add("BenchmarkTools")
Pkg.activate("../..")  # Activate the project environment
Pkg.instantiate()      # Install any missing dependencies
Pkg.status()          # Check if MyMlp is listed

# Now try importing
using BenchmarkTools
using LinearAlgebra

   Resolving package versions...
      Compat entries added for 
  No Changes to `~/Repos/AWiD/MyMlp/Project.toml`
  No Changes to `~/Repos/AWiD/MyMlp/Manifest.toml`
  Activating project at `~/Repos/AWiD/MyMlp`


Project MyMlp v0.1.0
Status `~/Repos/AWiD/MyMlp/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0


## Comparison of different optimizations to Variable

In [1]:
import Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

abstract type GraphNode end
abstract type Operator <: GraphNode end

# Original implementation
mutable struct VariableOriginal <: GraphNode
    output :: Any
    grad :: Any
    name :: String
    VariableOriginal(output; name="?") = new(output, nothing, name)
end

# Optimized implementation
mutable struct VariableOptimized{T<:Float64} <: GraphNode
    output :: T
    grad :: Union{Nothing, T}
    name :: String
    VariableOptimized(output::T; name="?") where T<:Float64 = new{T}(output, nothing, name)
end

# RefValue-based immutable implementation
struct VariableRef <: GraphNode
    output :: Base.RefValue{Float64}
    grad   :: Union{Nothing, Base.RefValue{Float64}}
    name   :: String
    function VariableRef(output::Float64; name="?")
        new(Base.RefValue(output), nothing, name)
    end
end

# Functions to benchmark: reading
function read_output(v::VariableOriginal)
    v.output + 1.0
end

function read_output(v::VariableOptimized)
    v.output + 1.0
end

function read_output(v::VariableRef)
    v.output[] + 1.0
end

# Functions to benchmark: writing
function write_output!(v::VariableOriginal)
    v.output = v.output + 1.0
end

function write_output!(v::VariableOptimized)
    v.output = v.output + 1.0
end

function write_output!(v::VariableRef)
    v.output[] = v.output[] + 1.0
end

# Create instances
v1 = VariableOriginal(1.0)
v2 = VariableOptimized(1.0)
v3 = VariableRef(1.0)

# Benchmark READ
println("### Benchmark: READ access ###")
println("Original:")
@btime read_output($v1)

println("Optimized:")
@btime read_output($v2)

println("RefValue:")
@btime read_output($v3)

# Benchmark WRITE
println("\n### Benchmark: WRITE mutation ###")
println("Original:")
@btime write_output!($v1)

println("Optimized:")
@btime write_output!($v2)

println("RefValue:")
@btime write_output!($v3)



   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


### Benchmark: READ access ###
Original:
  14.034 ns (1 allocation: 16 bytes)
Optimized:
  1.575 ns (0 allocations: 0 bytes)
RefValue:
  1.793 ns (0 allocations: 0 bytes)

### Benchmark: WRITE mutation ###
Original:
  14.417 ns (1 allocation: 16 bytes)
Optimized:
  2.013 ns (0 allocations: 0 bytes)
RefValue:
  2.013 ns (0 allocations: 0 bytes)


500503.0

## Next

In [ ]:
import Base: *, +
import LinearAlgebra: mul!

abstract type GraphNode end
abstract type Operator <: GraphNode end

# Definition of basic structures for computational graph
struct Constant{T<:Float64} <: GraphNode
    output :: T
end

mutable struct Variable <: GraphNode
    output :: Matrix{Float64}
    gradient :: Union{Nothing, Matrix{Float64}}
    name :: String
    Variable(output::Matrix{Float64}; name="?") = new(output, nothing, name)
end

mutable struct ScalarOperator{F} <: Operator
    inputs :: Union{Nothing, Tuple{GraphNode, GraphNode}}
    output :: Union{Nothing, Float64}
    gradient :: Union{Nothing, Float64}
    name :: String
    ScalarOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end

mutable struct BroadcastedOperator{F} <: Operator
    inputs :: Union{Nothing, Tuple{GraphNode, GraphNode}, Tuple{GraphNode}}
    output :: Union{Nothing, Matrix{Float64}}
    gradient :: Union{Nothing, Matrix{Float64}}
    name :: String
    BroadcastedOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end


import Base: show, summary
show(io::IO, x::ScalarOperator{F}) where {F} = print(io, "op ", x.name, "(", F, ")");
show(io::IO, x::BroadcastedOperator{F}) where {F} = print(io, "op.", x.name, "(", F, ")");
show(io::IO, x::Constant) = print(io, "const ", x.output)
show(io::IO, x::Variable) = begin
    print(io, "var ", x.name);
    print(io, "\n ┣━ ^ "); summary(io, x.output)
    print(io, "\n ┗━ ∇ ");  summary(io, x.gradient)
end


function visit(node::GraphNode, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        push!(order, node)
    end
    return nothing
end

function visit(node::Operator, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        for input in node.inputs
            visit(input, visited, order)
        end
        push!(order, node)
    end
    return nothing
end

function topological_sort(head::GraphNode)
    visited = Set()
    order = Vector()
    visit(head, visited, order)
    return order
end


# x * y (aka matrix multiplication)
*(A::GraphNode, x::GraphNode) = BroadcastedOperator(mul!, A, x)
forward(::BroadcastedOperator{typeof(mul!)}, A, x) = return A * x
backward(::BroadcastedOperator{typeof(mul!)}, A, x, g) = tuple(g * x', A' * g)

# relu activation
relu(x::GraphNode) = BroadcastedOperator(relu, x)
forward(::BroadcastedOperator{typeof(relu)}, x) = return x .* (x .> 0.0)
backward(::BroadcastedOperator{typeof(relu)}, x, g) = tuple(g .* (x .> 0.0), nothing)

# add operation (for bias)
+(x::GraphNode, y::GraphNode) = BroadcastedOperator(+, x, y)
forward(::BroadcastedOperator{typeof(+)}, x, y) = return x .+ y
backward(::BroadcastedOperator{typeof(+)}, x, y, g) = tuple(g, g)

# sigmoid activation
σ(x::GraphNode) = BroadcastedOperator(σ, x)
forward(::BroadcastedOperator{typeof(σ)}, x) = return 1.0 ./ (1.0 .+ exp.(-x))
backward(node::BroadcastedOperator{typeof(σ)}, x, g) = let
    y = node.output
    # vec() przeksztalca macierz w wektor, wymagane do dzialania
    J = diagm(vec(y .* (1.0 .- y)))
    tuple(J' * g)
end

# binarycrossentropy
binarycrossentropy(y::GraphNode, ŷ::GraphNode) = ScalarOperator(binarycrossentropy, y, ŷ)
forward(::ScalarOperator{typeof(binarycrossentropy)}, ŷ, y) = begin
    return -mean(y .* log.(ŷ) + (1.0 .- y) .* log.(1.0 .- ŷ))
end
backward(::ScalarOperator{typeof(binarycrossentropy)}, ŷ, y, g) = begin
    J = (ŷ .- y) ./ (ŷ .* (1.0 .- ŷ))
    return (J * g, nothing)
end

backward (generic function with 5 methods)

In [3]:
reset!(node::Constant) = nothing

reset!(node::Variable) = node.gradient = nothing
reset!(node::Operator) = node.gradient = nothing

compute!(node::Constant) = nothing
compute!(node::Variable) = nothing

compute!(node::Operator) =
    node.output = forward(node, [input.output for input in node.inputs]...)

function forward!(order::Vector)
    #   Iteruje przez każdy węzeł w order.
    for node in order
        compute!(node)
        reset!(node)
    end
    return last(order).output
end

forward! (generic function with 1 method)

In [4]:
update!(node::Constant, gradient) = nothing

update!(node::GraphNode, gradient) = if isnothing(node.gradient)
    node.gradient = gradient else node.gradient .+= gradient
end


function backward!(order::Vector; seed=1.0)
    result = last(order)   #   The output node

    if isa(result.output, Matrix{Float64})
        result.gradient = ones(Float64, size(result.output))
    else
        result.gradient = seed
        @assert length(result.output) == 1 "Gradient is defined only for scalar functions"
    end

    for node in reverse(order)   #   Iterate through nodes in reverse topological order.
        backward!(node)   #   Compute and propagate gradients backwards.
    end
    return nothing
end

function backward!(node::Constant) end
function backward!(node::Variable) end

function backward!(node::Operator)
    inputs = node.inputs

    gradients = backward(node, [input.output for input in inputs]..., node.gradient)

    for (input, gradient) in zip(inputs, gradients)
        update!(input, gradient)
    end
    return nothing
end

backward! (generic function with 4 methods)

## Test wejścia do neuronu

In [12]:
x = Variable(reshape([1.0, 2.0, 3.0], 3, 1), name="x")

var x
 ┣━ ^ 3×1 Matrix{Float64}
 ┗━ ∇ Nothing

In [13]:
w = Variable(reshape([1.0, 2.0, 3.0], 1, 3), name="w")

var w
 ┣━ ^ 1×3 Matrix{Float64}
 ┗━ ∇ Nothing

In [14]:
z = w * x
z.name = "z"

"z"

In [15]:
order = topological_sort(z)
println("Topological order:")
order


Topological order:


3-element Vector{Any}:
 var w
 ┣━ ^ 1×3 Matrix{Float64}
 ┗━ ∇ Nothing
 var x
 ┣━ ^ 3×1 Matrix{Float64}
 ┗━ ∇ Nothing
 op.z(typeof(mul!))

In [16]:
y = forward!(order)
z.output

1×1 Matrix{Float64}:
 14.0

In [17]:
backward!(order)
w.gradient

1×3 Matrix{Float64}:
 1.0  2.0  3.0

## Test 2 szeregowych Neuronów - 1 warstwa + bias


In [ ]:
x = Variable(reshape([1.0, 2.0, 3.0], 3, 1), name="x")
w = Variable(reshape([-2.0, 3.0, -4.0, 5.0, -6.0, 7.0], 2, 3), name="w")
z = w * x
z.name = "z"
c = Constant(1.0)
d = z + c
# dense_layer_2 = σ(z)
# dense_layer_2.name = "σ(z)"
dense_layer_2 = relu(d)
dense_layer_2.name = "relu(d)"
order = topological_sort(dense_layer_2)
y = forward!(order)
backward!(order)

## Test 2. warstw neuronów 2-4

In [ ]:
#   Pierwsza warstwa
x = Variable(reshape([1.0, 2.0, 3.0], 3, 1), name="x")
w = Variable(reshape([2.0, 3.0, 4.0, 5.0, 6.0, 7.0], 2, 3), name="w1")
a = w * x
a.name = "a"
b = relu(a)
b.name = "b"

#   Druga warstwa
w2 = Variable(reshape([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], 4,2), name="w2")
c = w2 * b
c.name = "c"
d = relu(c)
d.name = "d"
order = topological_sort(d)

7-element Vector{Any}:
 var w2
 ┣━ ^ 4×2 Matrix{Float64}
 ┗━ ∇ Nothing
 var w1
 ┣━ ^ 2×3 Matrix{Float64}
 ┗━ ∇ Nothing
 var x
 ┣━ ^ 3×1 Matrix{Float64}
 ┗━ ∇ Nothing
 op.a(typeof(mul!))
 op.b(typeof(relu))
 op.c(typeof(mul!))
 op.d(typeof(relu))

In [12]:
ŷ = forward!(order)
backward!(order)

## Test binary cross entropy

In [5]:
ŷ = Variable(reshape([0.8], 1, 1), name="ŷ")
y = Variable(reshape([1.0], 1, 1), name="y")
loss = binarycrossentropy(ŷ, y)
loss.name = "loss"
order = topological_sort(loss)
result = forward!(order)


0.2231435513142097

In [6]:
backward!(order)